In [1]:
import requests
from contextlib import closing
import csv
import json_stream.requests

In [2]:
url =  'https://zenodo.org/record/5887261'

# Transform URL for API request
record_id = url.split('/')[-1]
url = 'https://zenodo.org/api/records/' + record_id


In [3]:
try:
    r = requests.get(url, timeout=2) #timeout in seconds
except requests.exceptions.ConnectTimeout:
    print('Connection error')

if not r.ok:
    print('DOI could not be resolved. Try again, or use a different URL.')


In [4]:
if r.ok:
    js = r.json()
    files = js['files']
    total_size = sum(f['size'] for f in files)

In [5]:
# Get file description
print(js['metadata']['description'])

<p>Nomenclature des unit&eacute;s territoriales statistiques (NUTS) of Europe and UK at level 0, 1, 2 and 3 according to the European convention of 2021.</p>

<p>&nbsp;</p>

<p>&copy; EuroGeographics for the administrative boundaries</p>


In [6]:
total_size

48775784

In [7]:
for f in files:
    fname = f['key']
    link = 'https://zenodo.org/record/{}/files/{}'.format(
        record_id, fname
    ).replace(' ', '%20')
    size = "{:.2f}".format(f['size'] / 1024**2) + ' MB'
    print(fname, link, size)

Italian Municipality Codes.json https://zenodo.org/record/5887261/files/Italian%20Municipality%20Codes.json 5.81 MB
NUTS_RG_01M_2021_4326_LEVL_0.geojson https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_0.geojson 6.41 MB
NUTS_RG_01M_2021_4326_LEVL_1.geojson https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_1.geojson 8.13 MB
NUTS_RG_01M_2021_4326_LEVL_2.geojson https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_2.geojson 10.20 MB
NUTS_RG_01M_2021_4326_LEVL_3.geojson https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_3.geojson 15.97 MB


In [8]:
url = 'https://zenodo.org/record/3694065/files/var10_30yrMeanAnnualCumulatedGDDAbove5degreesC_rcp26_2041_2070.csv'
url = 'https://zenodo.org/record/3694065/files/var13_AnnualVariationSolarRadiation_historical_1971_2005.csv?download=1'

In [9]:
url = 'https://zenodo.org/record/6958310/files/espeni.csv?download=1'

In [10]:

# For CSV files
with closing(requests.get(url, stream=True)) as r:
    f = (line.decode("utf-8") for line in r.iter_lines())
    reader = csv.reader(f, delimiter=",", quotechar='"')
    for idx, row in enumerate(reader):
       print(row)
       if idx>2:
        break


['ELEXM_SETTLEMENT_DATE', 'ELEXM_SETTLEMENT_PERIOD', 'ELEXM_utc', 'ELEXM_localtime', 'ELEXM_ROWFLAG', 'NGEM_ROWFLAG', 'POWER_ESPENI_MW', 'POWER_ELEXM_CCGT_MW', 'POWER_ELEXM_OIL_MW', 'POWER_ELEXM_COAL_MW', 'POWER_ELEXM_NUCLEAR_MW', 'POWER_ELEXM_WIND_MW', 'POWER_ELEXM_PS_MW', 'POWER_ELEXM_NPSHYD_MW', 'POWER_ELEXM_OCGT_MW', 'POWER_ELEXM_OTHER_POSTCALC_MW', 'POWER_ELEXM_BIOMASS_POSTCALC_MW', 'POWER_NGEM_EMBEDDED_SOLAR_GENERATION_MW', 'POWER_NGEM_EMBEDDED_WIND_GENERATION_MW', 'POWER_NGEM_BRITNED_FLOW_MW', 'POWER_NGEM_EAST_WEST_FLOW_MW', 'POWER_NGEM_MOYLE_FLOW_MW', 'POWER_NGEM_NEMO_FLOW_MW', 'POWER_NGEM_IFA_FLOW_MW', 'POWER_NGEM_IFA2_FLOW_MW', 'POWER_NGEM_NSL_FLOW_MW']
['2008-11-05', '43', '2008-11-05T21:00:00+00:00', '2008-11-05T21:00:00+00:00', '1', '1', '44020', '16468', '0', '19525', '5646', '62', '580', '314', '0', '0', '0', '0', '129', '0', '0', '-59', '', '1355', '', '']
['2008-11-05', '44', '2008-11-05T21:30:00+00:00', '2008-11-05T21:30:00+00:00', '1', '1', '42169', '15240', '0', '18

In [11]:
url = 'https://zenodo.org/record/5887261/files/Italian%20Municipality%20Codes.json'
#url = 'https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_0.geojson?download=1'

In [12]:
# For json/geosjon files
# see https://pypi.org/project/json-stream/

# Coding this basic function took me sooo long because I'm retarded :'(
def fill_sample_dic(data, max_depth = 5, depth = 0):
    if depth>=max_depth:
        return None
    # Case we're dealing with a node with several labeled fields
    dic = {}
    if type(data) == json_stream.base.PersistentStreamingJSONObject:
        for k, v in data.items():
            dic[k] = fill_sample_dic(v, max_depth = max_depth, depth = depth+1)
        return dic
    # Case we're dealing with a node with several unlabeled fields
    elif type(data) == json_stream.base.PersistentStreamingJSONList:
        for idx, v in enumerate(data):
            dic[idx] = fill_sample_dic(v, max_depth = max_depth, depth = depth+1)
        return dic
    else:
        return data



# Build a sample dictionnary of the data
with requests.get(url, stream=True) as response:
    data = json_stream.requests.load(response, persistent=True)
    dic = fill_sample_dic(data, max_depth = 3)

print(dic)



{'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': None}}, 'features': {0: {'type': None, 'properties': None, 'geometry': None}, 1: {'type': None, 'properties': None, 'geometry': None}, 2: {'type': None, 'properties': None, 'geometry': None}, 3: {'type': None, 'properties': None, 'geometry': None}, 4: {'type': None, 'properties': None, 'geometry': None}, 5: {'type': None, 'properties': None, 'geometry': None}, 6: {'type': None, 'properties': None, 'geometry': None}, 7: {'type': None, 'properties': None, 'geometry': None}, 8: {'type': None, 'properties': None, 'geometry': None}, 9: {'type': None, 'properties': None, 'geometry': None}, 10: {'type': None, 'properties': None, 'geometry': None}, 11: {'type': None, 'properties': None, 'geometry': None}, 12: {'type': None, 'properties': None, 'geometry': None}, 13: {'type': None, 'properties': None, 'geometry': None}, 14: {'type': None, 'properties': None, 'geometry': None}, 15: {'type': None, 'properties': None, 'ge